# Visual odometry pipeline

### Import requirements

In [37]:
import cv2
import glob
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

%matplotlib ipympl

from ImageProcessor import ImageProcessor


### Load images and depth map

In [38]:
folderChoice = 0
path = "".join(["testImages/visualOdometryTestImages/", str(folderChoice)])

# Load set of top images
imageGlobT = sorted(glob.glob("".join([path, "/top_*", ".png"])))

# Load set of bottom images
imageGlobB = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))

# Load depth map
imageGlobD = sorted(glob.glob("".join([path, "/topDepth_*", ".png"])))

if not (len(imageGlobT)==len(imageGlobB) and \
                len(imageGlobB)==len(imageGlobD)):
    print("Images could not be matched")

print ("Selections: 0-{}".format(len(imageGlobT)-1))

Selections: 0-1


In [39]:
viewSelection = 0

# Top images
imageT0 = cv2.imread(imageGlobT[viewSelection])
imageT1 = cv2.imread(imageGlobT[viewSelection+1])

# Bottom images
imageB0 = cv2.imread(imageGlobB[viewSelection])
imageB1 = cv2.imread(imageGlobB[viewSelection+1])

# Depth maps; -1 flag to load them as is
depth0 = cv2.imread(imageGlobD[viewSelection], -1)
depth1 = cv2.imread(imageGlobD[viewSelection+1], -1)

### View top camera images

In [40]:
plt.figure()
plt.suptitle("Top images")
plt.imshow(cv2.cvtColor(np.hstack([imageT0, imageT1]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### View bottom camera images

In [41]:
plt.figure()
plt.suptitle("Bottom images")
plt.imshow(cv2.cvtColor(np.hstack([imageB0, imageB1]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### View depth images
Depth images correspond to top camera

In [42]:
plt.figure()
plt.suptitle("Depth maps")
plt.imshow(np.hstack([depth0, depth1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Feature Extraction


In [43]:
orb = cv2.ORB_create()
#def extract_features(img):
def extract_features(img1,img2):
    keyp1 = orb.detect(img1,None)
    keyp1,des1 = orb.compute(img1, keyp1)
    keyp2 = orb.detect(img2,None)
    keyp2,des2 = orb.compute(img2, keyp2)
    
    img1 = cv2.drawKeypoints(img1, keyp1, None, color=(0,255,0),flags=0)
    img2 = cv2.drawKeypoints(img2,keyp2,None,color=(0,255,0),flags=0) 
    plt.figure()
    # plt.imshow(img2), 
    plt.imshow(cv2.cvtColor(np.hstack([img1, img2]), cv2.COLOR_BGR2RGB))
    plt.show()
    return (keyp1,des1),(keyp2,des2)

In [44]:
orb1,orb2=extract_features(imageT0,imageT1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Feature matching


In [45]:
keyp1 = orb.detect(imageT0,None)
keyp1,des1 = orb.compute(imageT0, keyp1)
keyp2 = orb.detect(imageT1,None)
keyp2,des2 = orb.compute(imageT1, keyp2)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)
matches = sorted(matches, key = lambda x:x.distance)
img3 = cv2.drawMatches(imageT0,keyp1,imageT1,keyp2,matches[:10],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure()
plt.imshow(img3),plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.image.AxesImage at 0x1ea8c1edb20>, None)

Number of Matched features


In [46]:
print("Number of features matched in frames {0} and {1}: {2}",len(matches))

Number of features matched in frames {0} and {1}: {2} 250


Trajectory Estimation

In [47]:
def estimate_motion():
    imageProcessor=ImageProcessor()
    imageProcessor.loadMonoCalibration()
    k=imageProcessor.cameraMatrixL
    
    rmat=np.eye(3)
    tvec=np.zeros((3,1))
    image1_points = []
    image2_points = []

    for m in matches:
        train_idx = m.trainIdx
        query_idx = m.queryIdx

        p1x,p1y = keyp1[query_idx].pt 
        image1_points.append([p1x,p1y])

        p2x,p2y = keyp2[train_idx].pt 
        image2_points.append([p2x,p2y])
    E, mask = cv2.findEssentialMat(np.array(image1_points), np.array(image2_points), k) 
    retval,rmat,tvec,mask=cv2.recoverPose(E,np.array(image1_points),np.array(image2_points),k)
    return rmat,tvec,image1_points,image2_points      



In [48]:
i = 30
match = matches[i]
rmat, tvec, image1_points, image2_points = estimate_motion()

print("Estimated rotation:\n {0}".format(rmat))
print("Estimated translation:\n {0}".format(tvec))

Reading from data/monoCalibration.json
Loaded mono calibration
Estimated rotation:
 [[ 0.99445956 -0.03551991  0.09893699]
 [ 0.04459262  0.99484699 -0.09105477]
 [-0.09519291  0.09496215  0.99091902]]
Estimated translation:
 [[-0.65096869]
 [ 0.63698372]
 [ 0.41290617]]
